In [46]:
import emcee
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import os, sys, time, math
import torch
from tqdm import tqdm
from torch.multiprocessing import Pool#, set_start_method
from getdist import plots, MCSamples
import getdist

sys.path.append('/home/joeadamo/Research') #<- parent directory of dark emulator code
from DarkEmuPowerRSD import pkmu_nn, pkmu_hod
from CovNet import CovNet

In [47]:
plt.rcParams['figure.facecolor'] = 'white'
#try:
#    set_start_method('spawn')
#except:
#    pass

In [48]:
training_dir = "/home/joeadamo/Research/CovNet/Data/Training-Set/"
data_dir =  "/home/joeadamo/Research/CovNet/Data/"
PCA_dir = "/home/joeadamo/Research/CovNet/Data/PCA-Set/"
BOSS_dir = "/home/joeadamo/Research/Data/BOSS-DR12/"
CovaPT_dir = "/home/joeadamo/Research/CovaPT/Example-Data/"

In [49]:
cosmo_prior = np.array([[66.5, 75.5],
                        [0.10782, 0.13178],
                        [0.0211375, 0.0233625],
                        [1.1885e-9, 2.031e-9],#[2.4752, 3.7128],
                        [1.806, 2.04],
                        [-2.962, 0.458]])

cosmo_fid = np.array([70,0.1198,0.02225,2e-9,2.0,0.])

gparams = {'logMmin': 13.9383, 'sigma_sq': 0.7918725**2, 'logM1': 14.4857, 'alpha': 1.19196,  'kappa': 0.600692, 
          'poff': 0.0, 'Roff': 2.0, 'alpha_inc': 0., 'logM_inc': 0., 'cM_fac': 1., 'sigv_fac': 1., 'P_shot': 0.}
redshift = 0.5

In [50]:
P_BOSS = np.loadtxt(BOSS_dir+"Cl-BOSS-DR12.dat")
pgg = pkmu_hod()

data_vector = np.concatenate((P_BOSS[1], P_BOSS[2]))

def model_vector(params, gparams):
    """
    Calculates the model vector using Yosuke's galaxy power spectrum emulator
    """
    #print(params)
    h = params[0] / 100
    omch2 = params[1]
    ombh2 = params[2]
    #assert omch2 <= 0.131780
    As = np.log(1e10 * params[3])
    #assert As >= 2.47520
    ns = 0.965
    Om0 = (omch2 + ombh2 + 0.00064) / (h**2)
    
    # rebuild parameters into correct format (ombh2, omch2, 1-Om0, ln As, ns, w)
    cparams = np.array([ombh2, omch2, 1-Om0, As, ns, -1])
    redshift = 0.5
    k = np.linspace(0.005, 0.25, 50)
    mu = np.linspace(0.1,0.9,4)
    alpha_perp = 1.1
    alpha_para = 1

    pgg.set_cosmology(cparams, redshift)
    pgg.set_galaxy(gparams)
    P0_emu = pgg.get_pl_gg_ref(0, k, alpha_perp, alpha_para, name='total')
    P2_emu = pgg.get_pl_gg_ref(2, k, alpha_perp, alpha_para, name='total')
    return np.concatenate((P0_emu, P2_emu))

initialize cosmo_class
Initialize pklin emulator
initialize propagator emulator
Initialize sigma_d emulator
initialize cross-correlation emulator
initialize auto-correlation emulator
Initialize sigmaM emulator
initialize xinl emulator


In [51]:
def ln_prior(theta):
    for i in range(len(theta)):
        if (theta[i] < cosmo_prior[i,0]) or (theta[i] > cosmo_prior[i,1]):
            return -np.inf
    return 0.

def ln_lkl(theta, C):
    P = np.linalg.inv(C)
    x = model_vector(theta, gparams) - data_vector
    lkl = -0.5 * np.matmul(x.T, np.matmul(P, x))
    return lkl

def ln_prob(theta, C):
    p = ln_prior(theta)
    if p != -np.inf:
        return p * ln_lkl(theta, C)
    else: return p

In [52]:
#os.environ["OMP_NUM_THREADS"] = "8"

N_MCMC        = 4000
N_WALKERS     = 40
NDIM_SAMPLING = 6

theta0    = cosmo_fid
theta_std = np.array([1., 0.001, 0.0001, 0.01 * 2e-9, 0.01, 0.01])

# Starting position of the emcee chain
pos0 = theta0[np.newaxis] + theta_std[np.newaxis] * np.random.normal(size=(N_WALKERS, NDIM_SAMPLING))

C = np.loadtxt("/home/joeadamo/Research/Data/CovA-survey.txt")

In [53]:
t1 = time.time()
#with Pool() as pool:
emu_sampler = emcee.EnsembleSampler(N_WALKERS, NDIM_SAMPLING, ln_prob, args=(C,))#, pool=pool)
emu_sampler.run_mcmc(pos0, N_MCMC, progress=True)
t2 = time.time()
print("Done!, took {:0.0f} minutes {:0.2f} seconds".format(math.floor((t2 - t1)/60), (t2 - t1)%60))

  1%|▎                                      | 27/4000 [05:16<10:53:14,  9.87s/it]Traceback (most recent call last):
  File "/home/joeadamo/virtual-enviornments/cov-env/lib/python3.8/site-packages/emcee/ensemble.py", line 624, in __call__
    return self.f(x, *self.args, **self.kwargs)
  File "/tmp/ipykernel_1925945/442118246.py", line 16, in ln_prob
    return p * ln_lkl(theta, C)
  File "/tmp/ipykernel_1925945/442118246.py", line 9, in ln_lkl
    x = model_vector(theta, gparams) - data_vector
  File "/tmp/ipykernel_1925945/1368483783.py", line 28, in model_vector
    pgg.set_cosmology(cparams, redshift)
  File "/home/joeadamo/Research/DarkEmuPowerRSD/src/pkmu_hod.py", line 33, in set_cosmology
    self.emu.set_cosmology(cparam)
  File "/home/joeadamo/virtual-enviornments/cov-env/lib/python3.8/site-packages/dark_emulator/darkemu/de_interface.py", line 73, in set_cosmology
    self.g1.set_cosmology(self.cosmo)
  File "/home/joeadamo/virtual-enviornments/cov-env/lib/python3.8/site-packag

emcee: Exception while calling your likelihood function:
  params: [7.04548109e+01 1.20430743e-01 2.14452673e-02 1.54343067e-09
 1.88081521e+00 3.82858377e-02]
  args: (array([[5.58098732e+08, 1.10266573e+08, 1.22716985e+07, ...,
        1.20949806e+03, 1.02803048e+03, 8.51760220e+02],
       [1.10266573e+08, 1.32667689e+08, 3.50894851e+07, ...,
        3.24963659e+03, 2.72538712e+03, 2.21185569e+03],
       [1.22716985e+07, 3.50894851e+07, 6.04526154e+07, ...,
        4.64662343e+03, 3.89224164e+03, 3.15124621e+03],
       ...,
       [1.20949806e+03, 3.24963659e+03, 4.64662343e+03, ...,
        1.46303973e+04, 5.95207307e+03, 8.77478896e+02],
       [1.02803048e+03, 2.72538712e+03, 3.89224164e+03, ...,
        5.95207307e+03, 1.35403130e+04, 5.44534375e+03],
       [8.51760220e+02, 2.21185569e+03, 3.15124621e+03, ...,
        8.77478896e+02, 5.44534375e+03, 1.25672536e+04]]),)
  kwargs: {}
  exception:


KeyboardInterrupt: 

In [ ]:
#def plot_emcee_chain(sampler, N):
fig, ax = plt.subplots(NDIM_SAMPLING,1,figsize=(10, 2.5 * NDIM_SAMPLING))
for i in range(NDIM_SAMPLING):
    ax[i].plot(emu_sampler.chain[:,:,i].T, 'k-', lw=0.3)
plt.show()  

In [ ]:
#plot_emcee_chain(emu_sampler, NDIM_SAMPLING)

In [ ]:
N_BURN_IN = 500
N_THIN    = 15

samples = emu_sampler.chain[:,N_BURN_IN::N_THIN].reshape((-1,NDIM_SAMPLING))

In [ ]:
def plot_cosmo_contours(sample_list, legend_labels):
    names = ['H0', 'omegac', 'omegab', 'As', 'b1', 'b2']
    labels =  ["H_0", '\omega_c', '\omega_b', 'A_s', 'b1', 'b2']
    
    cosmo_truth = [69., 0.1199772, 0.0228528, 2e-9, 1.4, 0.]
    truth_dict = {}
    for name, truth in zip(names, cosmo_truth):
        truth_dict[name] = truth
        
    getdist_samples = []
    for samples, legend_label in zip(sample_list, legend_labels):
        cosmo_samples = samples[:,:6]
        getdist_samples.append(MCSamples(samples=cosmo_samples,names = names, labels=labels, label=legend_label))
    
    g = plots.get_subplot_plotter()
    g.triangle_plot(getdist_samples, filled=True, markers=truth_dict)
    
    plt.show()

In [ ]:
plot_cosmo_contours([samples], ['Tempered posterior'])